### 히스토그램의 최대 넓이 찾기

#### 1. 분할정복 활용하기
- 주어진 리스트에서 **가장 낮은 높이를 기준으로 좌우로 자른다**
- 이 때 분리되는 경우는 3가지이다.
    1. 가장 낮은 높이의 왼쪽까지(`[:idx]`)
    2. 가장 낮은 높이 + 1부터 오른쪽 끝까지
    3. 가장 낮은 높이 * 현재 리스트의 길이
    - 이 3가지 경우의 수 중 최댓값이 반드시 있음
    
- 문제는 `RecursionError`가 발생한다는 것
    - 이를 해결하는 방법으로 **세그먼트 트리**가 있다고 함 <- 없어도 풀 수는 있다.

- [분할 정복 풀이 참고](https://as-j.tistory.com/98) 
- 여기서 중요한 건 분할 정복 그 자체보다는 어떻게 최댓값을 갱신해나갈까에 있음
```
[대충 논리]
0. n은 "현재 들어온 리스트의 길이"이다.
1. mid값을 정함. l과 r은 mid -1과 mid로 정한다.
2. 이 l과 r은 현재 리스트에서 가운데 두 사각형 넓이부터 시작하기 위해 지정된 값이다. l과 r은 인접했기 떄문에 이 중 작은 높이값 * 2의 넓이부터 시작할 수 있다.
    - 여기서 이 작은 높이값을 min_h로 지정한다. 리스트의 범위를 다르게 줌으로써 분할 정복 과정에서 최소 높잇값은 달라질 수 있음
3. l이 0 이상, r이 n-1 이하인 동안 밑변을 넓혀나간다. heights[l-1]과 height[r+1]의 값을 비교해서, 이 중 "큰 값"과 "min_h"를 비교해 그 중 더 작은 값을 min_h로 갱신한다.
    - 이 때 큰 값을 고르는 이유는 최댓값을 찾아야 하기 때문이다. min_h가 4인데 heights[l-1] = 3, heights[r+1] = 5라고 생각해보자. 바로 알 수 있음
    - 또한, 선택된 방향으로 인덱스를 1칸 옮겨주고 밑변도 1을 넓혀준다
    - 그리고 직사각형 넓이를 계산한다
4. 마지막으로 이전 재귀식에서의 최댓값들과 현재 재귀식에서 구한 최댓값을 비교해 더 큰 값을 선택한다
```


In [62]:
import sys
sys.stdin = open('test.txt', 'r')
input = sys.stdin.readline

def func(heights: list):
    N = len(heights)
    if N == 1:
        return heights[0]
    mid = N // 2
    
    l = mid - 1
    r = mid 
    underbar = 2
    min_h = min(heights[l], heights[r])
    max_area = min_h * underbar
    
    while 0 <= l and r <= N - 1:
        l_move = heights[l-1] if l >= 1 else 0  
        r_move = heights[r+1] if r < N-1 else 0 
        if l_move > r_move:
            min_h = min(min_h, l_move)
            l -= 1   
        else:
            min_h = min(min_h, r_move)
            r += 1
        underbar += 1
        max_area = max(max_area, min_h * underbar)
    
    return max(func(heights[:mid]), func(heights[mid:]), max_area)
    
        


while True:
    inp = list(map(int, input().split()))
    if inp[0] == 0:
        break

    print(func(inp[1:]))

8
4000


In [ ]:
# Recursion Error가 뜬다
# def func(left, right):
#     """여기서 right는 마지막 인덱스 + 1"""
#     length = right - left
    
#     if length == 1:
#         return heights[left]
    
    
#     # 1. 최소값을 갖는 인덱스를 찾는다
#     min_value = 1e10 + 1
#     min_idx = -1
#     for idx in range(left, right):
#         if heights[idx] < min_value:
#             min_value = heights[idx]
#             min_idx = idx
    
#     # 2. 최소 인덱스를 기준으로 좌우를 나눈다
#     if left != min_idx:
#         left_lst_value = func(left, min_idx)
#     if right != min_idx:
#         right_lst_value = func(min_idx + 1, right)
#     min_box = min_value * length
    
#     return max(left_lst_value, right_lst_value, min_box)

### 2. 스택 활용하기
0. 기존 리스트의 마지막에 0 값을 append함
    - 높이 = 1을 갖는 값까지 구하기 위해
1. 스택을 구현함 : `[[0,0]]`부터 시작
2. 반복문을 돌리면서 기본적으로 현재 인덱스와 값을 스택에 푸시함
3. `현재 인덱스의 값 < 스택 top의 값`일 때 2.를 하기 전에 넓이 계산을 시작함
    1. 스택을 pop함
    2. pop한 값(value)과, 바로 아래에 있는 value가 동일한 동안 계속 pop 함
        - 작은 값을 만나면 이보다 큰 값은 스택에서 pop되므로, 이 결과 남은 stack의 값은 현재 인덱스의 값보다 크고, 최초 스택에서 pop된 값보단 작음
    3. 현재 인덱스 값은 직사각형에 포함되지 않는다. 가로 길이는 (현재 인덱스 - 1)에서 (스택 top의 인덱스)를 빼준 값이 됨. 세로는 pop한 value가 된다.
    4. 최댓값을 갱신한다.
- 이 넓이 계산은 스택 top의 값이 현재 인덱스의 값보다 큰 동안 계속 반복됨
- 위에서 언급한 최초 스택 pop 값보다 작고 현재 인덱스의 값보다 큰 경우들을 찾게 되는 것

In [24]:
import sys
sys.stdin = open('test.txt', 'r')
input = sys.stdin.readline

while True:
    inp = input().strip()
    
    if inp == '0':
        break
   
    lst = list(map(int, inp.split())) # 길이 n+1
    lst.append(0) # 길이 n+2
    n = lst[0] 
    
    stack = [[0, 0]]
    area = 0
    for i in range(1, n+2): # 0번 인덱스는 n값이니까 제외
        
        # push하기 전, push할 값이 top 값보다 작으면 면적을 계산한다.
        while lst[i] < stack[-1][1]:
            temp_lst = stack.pop()
            temp_area = 0
            
            while stack[-1][1] == temp_lst[1]:
                stack.pop()
            
            temp_area = ((i - 1) - stack[-1][0]) * temp_lst[1]
            area = max(temp_area, area)
        # 기본적으로 push는 계속 해줌
        stack.append([i, lst[i]])
        
    print(area)

8
4000


### 스택 쓰는 거 최초에 생각한 사람은 진짜 개 천재일거임

In [32]:
# 230226 스택 활용 복습

import sys
sys.stdin = open('test.txt', 'r')
input = sys.stdin.readline

while True:
    inp = list(map(int, input().split()))
    N = inp[0] 
    if N == 0:
        break
    
    inp.append(0)

    stack = [[0, 0]]
    max_area = -1

    for i in range(1, N+2):
        
        while inp[i] < stack[-1][1]:
            
            temp_height = stack.pop()
            temp_area = 0
            
            while stack[-1][1] == temp_height[1]:
                stack.pop()
            temp_area = ((i - 1) - stack[-1][0]) * temp_height[1]
            max_area = max(max_area, temp_area)
        stack.append([i, inp[i]])
    print(max_area)

8
4000


![으어](겁나긴직사각형.jpg)